In [1]:
spark

In [2]:
# !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1YfpOKU_vz4CJCjAwhwM7dFYfEJWZudme' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1YfpOKU_vz4CJCjAwhwM7dFYfEJWZudme" -O clicks2.dat.gz && rm -rf /tmp/cookies.txt

In [3]:
# !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1Jat5OLvFt5H4yF-Dw42bAQ1EZmQqQD-D' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1Jat5OLvFt5H4yF-Dw42bAQ1EZmQqQD-D" -O buys.dat.gz && rm -rf /tmp/cookies.txt

In [4]:
# !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1Ll7rqusi5ISuGWSpa6WGqmFaZcWUpR91' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1Ll7rqusi5ISuGWSpa6WGqmFaZcWUpR91" -O test.dat.gz && rm -rf /tmp/cookies.txt

## Loading yoochoose-buys.dat

In [2]:
df_buys = (spark.read.format('csv')
.options(inferSchema=True)
.load('recsys2015/buys.dat.gz')
.toDF('sess_id', 'timestamp', 'item_id', 'price', 'qty')).cache()

In [138]:
df_buys.show(5)

+-------+--------------------+---------+-----+---+
|sess_id|           timestamp|  item_id|price|qty|
+-------+--------------------+---------+-----+---+
| 420374|2014-04-06 18:44:...|214537888|12462|  1|
| 420374|2014-04-06 18:44:...|214537850|10471|  1|
| 281626|2014-04-06 09:40:...|214535653| 1883|  1|
| 420368|2014-04-04 06:13:...|214530572| 6073|  1|
| 420368|2014-04-04 06:13:...|214835025| 2617|  1|
+-------+--------------------+---------+-----+---+
only showing top 5 rows



In [7]:
df_buys.printSchema()

root
 |-- sess_id: integer (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- item_id: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- qty: integer (nullable = true)



In [8]:
df_buys.storageLevel.useMemory

True

## Loading yoochoose-clicks.dat

In [3]:
df_clicks = (spark.read.format('csv')
.options(inferSchema=True)
.load('recsys2015/clicks.dat.gz')
.toDF('sess_id', 'timestamp', 'item_id', 'category')).cache()

In [10]:
df_clicks.show(5)

+-------+--------------------+---------+--------+
|sess_id|           timestamp|  item_id|category|
+-------+--------------------+---------+--------+
|      1|2014-04-07 10:51:...|214536502|       0|
|      1|2014-04-07 10:54:...|214536500|       0|
|      1|2014-04-07 10:54:...|214536506|       0|
|      1|2014-04-07 10:57:...|214577561|       0|
|      2|2014-04-07 13:56:...|214662742|       0|
+-------+--------------------+---------+--------+
only showing top 5 rows



In [4]:
from pyspark.sql.types import IntegerType
df_clicks = df_clicks.withColumn("category", df_clicks["category"].cast(IntegerType()))

In [12]:
df_clicks.printSchema()

root
 |-- sess_id: integer (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- item_id: integer (nullable = true)
 |-- category: integer (nullable = true)



In [8]:
# df_clicks.storageLevel.useMemory

## Question 1

How many records in yoochoose-clicks.dat and yoochoose-buys.dat?

In [34]:
df_clicks.count(), df_buys.count()

(33003944, 1150753)

## Question 2

How many columns are there in the above files and what are the column types? Confirm that it matches with data description.

In [36]:
len(df_clicks.columns)

4

In [37]:
df_clicks.printSchema()

root
 |-- sess_id: integer (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- item_id: integer (nullable = true)
 |-- category: string (nullable = true)



In [38]:
len(df_buys.columns)

5

In [39]:
df_buys.printSchema()

root
 |-- sess_id: integer (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- item_id: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- qty: integer (nullable = true)



Column types match data description.

## Question 3

Convert the above files into parquet for more efficient reading.

In [44]:
df_buys.write.format('parquet').mode("overwrite").save('recsys2015/buys')

In [45]:
df_clicks.write.format('parquet').mode("overwrite").save('recsys2015/clicks')

## Question 4

What is the min and max timestamp for buys and clicks?

In [5]:
import pyspark.sql.functions as F

For buys:

In [49]:
df_buys.select(F.min('timestamp')).collect(), df_buys.select(F.max('timestamp')).collect()

[Row(min(timestamp)=datetime.datetime(2014, 4, 1, 3, 5, 31, 743000))]

For clicks:

In [50]:
df_clicks.select(F.min('timestamp')).collect(), df_clicks.select(F.max('timestamp')).collect()

([Row(min(timestamp)=datetime.datetime(2014, 4, 1, 3, 0, 0, 124000))],
 [Row(max(timestamp)=datetime.datetime(2014, 9, 30, 2, 59, 59, 430000))])

## Question 5

Is any session_id present in the buys dataset  but not in clicks dataset?

In [52]:
df_buys.select('sess_id').subtract(df_clicks.select('sess_id')).collect()

[]

No.

## Question 6

If you struggle with the size of the data, take a sample of 100K samples from the clicks dataset.

In [63]:
df_clicks_small = df_clicks.limit(100000)

## Question 7

Find the number of null values for each column?

In [60]:
def count_missings(spark_df):
    df = spark_df.select([F.count(F.when(F.isnan(c) | F.isnull(c), c)).alias(c) for (c,c_type) in spark_df.dtypes if c_type not in ('timestamp', 'string', 'date')]).toPandas()
    return df

In [61]:
count_missings(df_clicks_small)

,sess_id,item_id
0,0,0


In [62]:
count_missings(df_buys)

,sess_id,item_id,price,qty
0,0,0,0,0


No null values present.

## Question 8

Find the normalized frequency (frequency / total) of each category in the clicks dataset. Add a new column ("cat_freq") that shows the normed frequency for each row associated with each category.

In [64]:
gdf_clicks = df_clicks_small.groupBy('category')

In [65]:
counts = gdf_clicks.agg(F.count('item_id')).collect()

In [66]:
count_per_cat = {}
for c in counts:
    count_per_cat[c[0]] = c[1]

In [67]:
total_clicks = df_clicks_small.count()

In [68]:
from pyspark.sql.types import FloatType
freq = F.udf(lambda category: count_per_cat[category]/total_clicks, FloatType())

In [69]:
df_clicks_small = df_clicks_small.withColumn('cat_freq', freq(df_clicks_small.category))
df_clicks_small.show(5)

+-------+--------------------+---------+--------+--------+
|sess_id|           timestamp|  item_id|category|cat_freq|
+-------+--------------------+---------+--------+--------+
|      1|2014-04-07 10:51:...|214536502|       0|     1.0|
|      1|2014-04-07 10:54:...|214536500|       0|     1.0|
|      1|2014-04-07 10:54:...|214536506|       0|     1.0|
|      1|2014-04-07 10:57:...|214577561|       0|     1.0|
|      2|2014-04-07 13:56:...|214662742|       0|     1.0|
+-------+--------------------+---------+--------+--------+
only showing top 5 rows



## Question 9

In the clicks dataset, create a new column special_offer (boolean). Set to TRUE if the value in the "category" column is 999.

In [70]:
df_clicks_small.printSchema()

root
 |-- sess_id: integer (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- item_id: integer (nullable = true)
 |-- category: integer (nullable = true)
 |-- cat_freq: float (nullable = true)



In [71]:
# from pyspark.sql.types import IntegerType
# df_clicks = df_clicks.withColumn("category", df_clicks["category"].cast(IntegerType()))

In [72]:
is999 = F.udf(lambda category: category == 999)

In [73]:
df_clicks_small = df_clicks_small.withColumn('special_offer', is999(df_clicks_small.category))

In [74]:
df_clicks_small.show(5)

+-------+--------------------+---------+--------+--------+-------------+
|sess_id|           timestamp|  item_id|category|cat_freq|special_offer|
+-------+--------------------+---------+--------+--------+-------------+
|      1|2014-04-07 10:51:...|214536502|       0|     1.0|        false|
|      1|2014-04-07 10:54:...|214536500|       0|     1.0|        false|
|      1|2014-04-07 10:54:...|214536506|       0|     1.0|        false|
|      1|2014-04-07 10:57:...|214577561|       0|     1.0|        false|
|      2|2014-04-07 13:56:...|214662742|       0|     1.0|        false|
+-------+--------------------+---------+--------+--------+-------------+
only showing top 5 rows



## Question 10

In the clicks dataset, create a new column missing_category (boolean). Set to TRUE if the value in the "category" column is 0.

In [75]:
missing = F.udf(lambda category: category == 0)

In [76]:
df_clicks_small = df_clicks_small.withColumn('missing_category', missing(df_clicks_small.category))

In [77]:
df_clicks_small.printSchema()

root
 |-- sess_id: integer (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- item_id: integer (nullable = true)
 |-- category: integer (nullable = true)
 |-- cat_freq: float (nullable = true)
 |-- special_offer: string (nullable = true)
 |-- missing_category: string (nullable = true)



In [78]:
df_clicks_small.show(5)

+-------+--------------------+---------+--------+--------+-------------+----------------+
|sess_id|           timestamp|  item_id|category|cat_freq|special_offer|missing_category|
+-------+--------------------+---------+--------+--------+-------------+----------------+
|      1|2014-04-07 10:51:...|214536502|       0|     1.0|        false|            true|
|      1|2014-04-07 10:54:...|214536500|       0|     1.0|        false|            true|
|      1|2014-04-07 10:54:...|214536506|       0|     1.0|        false|            true|
|      1|2014-04-07 10:57:...|214577561|       0|     1.0|        false|            true|
|      2|2014-04-07 13:56:...|214662742|       0|     1.0|        false|            true|
+-------+--------------------+---------+--------+--------+-------------+----------------+
only showing top 5 rows



## Question 11

Find the number of clicks in each session. Add a new column - "clicks_count" in the clicks dataset, show the count associated with each session. 

In [79]:
gdf_clicks = df_clicks_small.groupBy('sess_id')

In [80]:
counts = gdf_clicks.agg(F.count('item_id')).collect()

In [81]:
count_per_sess = {}
for c in counts:
    count_per_sess[c[0]] = c[1]

In [82]:
freq = F.udf(lambda sess_id: count_per_sess[sess_id])

In [83]:
df_clicks_small = df_clicks_small.withColumn('clicks_count', freq(df_clicks_small.sess_id))
df_clicks_small.show(5)

+-------+--------------------+---------+--------+--------+-------------+----------------+------------+
|sess_id|           timestamp|  item_id|category|cat_freq|special_offer|missing_category|clicks_count|
+-------+--------------------+---------+--------+--------+-------------+----------------+------------+
|      1|2014-04-07 10:51:...|214536502|       0|     1.0|        false|            true|           4|
|      1|2014-04-07 10:54:...|214536500|       0|     1.0|        false|            true|           4|
|      1|2014-04-07 10:54:...|214536506|       0|     1.0|        false|            true|           4|
|      1|2014-04-07 10:57:...|214577561|       0|     1.0|        false|            true|           4|
|      2|2014-04-07 13:56:...|214662742|       0|     1.0|        false|            true|           6|
+-------+--------------------+---------+--------+--------+-------------+----------------+------------+
only showing top 5 rows



## Question 12

Similarly, add "items_count" and "cat_count" for each session
    1. items_count - number of items per session
    2. cat_count - number of items per session


In [84]:
counts = gdf_clicks.agg(F.countDistinct('item_id')).collect()

In [85]:
count_per_sess = {}
for c in counts:
    count_per_sess[c[0]] = c[1]

In [86]:
freq = F.udf(lambda sess_id: count_per_sess[sess_id])

In [87]:
df_clicks_small = df_clicks_small.withColumn('items_count', freq(df_clicks_small.sess_id))
df_clicks_small.show(5)

+-------+--------------------+---------+--------+--------+-------------+----------------+------------+-----------+
|sess_id|           timestamp|  item_id|category|cat_freq|special_offer|missing_category|clicks_count|items_count|
+-------+--------------------+---------+--------+--------+-------------+----------------+------------+-----------+
|      1|2014-04-07 10:51:...|214536502|       0|     1.0|        false|            true|           4|          4|
|      1|2014-04-07 10:54:...|214536500|       0|     1.0|        false|            true|           4|          4|
|      1|2014-04-07 10:54:...|214536506|       0|     1.0|        false|            true|           4|          4|
|      1|2014-04-07 10:57:...|214577561|       0|     1.0|        false|            true|           4|          4|
|      2|2014-04-07 13:56:...|214662742|       0|     1.0|        false|            true|           6|          5|
+-------+--------------------+---------+--------+--------+-------------+--------

In [88]:
counts = gdf_clicks.agg(F.countDistinct('category')).collect()

In [89]:
count_per_sess = {}
for c in counts:
    count_per_sess[c[0]] = c[1]

In [90]:
freq = F.udf(lambda sess_id: count_per_sess[sess_id])

In [91]:
df_clicks_small = df_clicks_small.withColumn('items_count', freq(df_clicks_small.sess_id))
df_clicks_small.show(5)

+-------+--------------------+---------+--------+--------+-------------+----------------+------------+-----------+
|sess_id|           timestamp|  item_id|category|cat_freq|special_offer|missing_category|clicks_count|items_count|
+-------+--------------------+---------+--------+--------+-------------+----------------+------------+-----------+
|      1|2014-04-07 10:51:...|214536502|       0|     1.0|        false|            true|           4|          1|
|      1|2014-04-07 10:54:...|214536500|       0|     1.0|        false|            true|           4|          1|
|      1|2014-04-07 10:54:...|214536506|       0|     1.0|        false|            true|           4|          1|
|      1|2014-04-07 10:57:...|214577561|       0|     1.0|        false|            true|           4|          1|
|      2|2014-04-07 13:56:...|214662742|       0|     1.0|        false|            true|           6|          1|
+-------+--------------------+---------+--------+--------+-------------+--------

## Question 13

Clicks datasets: find  dwell time i.e. time gap between a click and previous click for each record

In [92]:
from pyspark.sql.window import Window

w = Window.partitionBy('sess_id').orderBy('timestamp')
df_clicks_small = (df_clicks_small.withColumn("previous_t", F.lag(df_clicks_small.timestamp, 1).over(w))
  .select('*', (F.unix_timestamp(df_clicks_small.timestamp) - F.unix_timestamp(F.col('previous_t'))).alias('dwell_time'))\
  )

In [93]:
df_clicks_small.show(5)

+-------+--------------------+---------+--------+--------+-------------+----------------+------------+-----------+--------------------+----------+
|sess_id|           timestamp|  item_id|category|cat_freq|special_offer|missing_category|clicks_count|items_count|          previous_t|dwell_time|
+-------+--------------------+---------+--------+--------+-------------+----------------+------------+-----------+--------------------+----------+
|    148|2014-04-01 16:06:...|214684093|       0|     1.0|        false|            true|           3|          1|                null|      null|
|    148|2014-04-01 16:10:...|214684093|       0|     1.0|        false|            true|           3|          1|2014-04-01 16:06:...|       253|
|    148|2014-04-01 16:18:...|214684093|       0|     1.0|        false|            true|           3|          1|2014-04-01 16:10:...|       489|
|    463|2014-04-04 08:25:...|214820385|       0|     1.0|        false|            true|           2|          1|    

## Question 14

### a. Find start time (start_ts) and end time  (end_ts) of each session

In [94]:
gdf_clicks = df_clicks_small.groupBy('sess_id')

In [95]:
df_sess = (df_clicks_small
.groupBy(F.col('sess_id'))
.agg(
    F.min('timestamp').alias('start_ts'),
    F.max('timestamp').alias('end_ts')
)).cache()

In [96]:
df_sess.show(5)

+-------+--------------------+--------------------+
|sess_id|            start_ts|              end_ts|
+-------+--------------------+--------------------+
|      1|2014-04-07 10:51:...|2014-04-07 10:57:...|
|      2|2014-04-07 13:56:...|2014-04-07 14:02:...|
|      3|2014-04-02 13:17:...|2014-04-02 13:30:...|
|      4|2014-04-07 12:09:...|2014-04-07 12:26:...|
|      6|2014-04-06 16:58:...|2014-04-06 17:02:...|
+-------+--------------------+--------------------+
only showing top 5 rows



### b. Find duration of each session in seconds (total_duration_secs)

Avoid zero values so as to avoid division by zero exception.

In [97]:
df_sess = (df_sess
.withColumn("total_duration_secs", F.unix_timestamp(F.col('end_ts')) - F.unix_timestamp(F.col('start_ts')) + 0.01))

In [98]:
df_sess.show(5)

+-------+--------------------+--------------------+-------------------+
|sess_id|            start_ts|              end_ts|total_duration_secs|
+-------+--------------------+--------------------+-------------------+
|      1|2014-04-07 10:51:...|2014-04-07 10:57:...|             351.01|
|      2|2014-04-07 13:56:...|2014-04-07 14:02:...|             359.01|
|      3|2014-04-02 13:17:...|2014-04-02 13:30:...|             746.01|
|      4|2014-04-07 12:09:...|2014-04-07 12:26:...|            1035.01|
|      6|2014-04-06 16:58:...|2014-04-06 17:02:...|             246.01|
+-------+--------------------+--------------------+-------------------+
only showing top 5 rows



### c. Find number of clicks per session (clicks_count_sum)

In [99]:
gdf_clicks = df_clicks_small.groupBy('sess_id')

In [100]:
counts = gdf_clicks.agg(F.count('item_id')).collect()

In [101]:
count_per_sess = {}
for c in counts:
    count_per_sess[c[0]] = c[1]

In [102]:
freq = F.udf(lambda sess_id: count_per_sess[sess_id], IntegerType())

In [103]:
df_sess = df_sess.withColumn('clicks_count_sum', freq(df_sess.sess_id))
df_sess.show(5)

+-------+--------------------+--------------------+-------------------+----------------+
|sess_id|            start_ts|              end_ts|total_duration_secs|clicks_count_sum|
+-------+--------------------+--------------------+-------------------+----------------+
|      1|2014-04-07 10:51:...|2014-04-07 10:57:...|             351.01|               4|
|      2|2014-04-07 13:56:...|2014-04-07 14:02:...|             359.01|               6|
|      3|2014-04-02 13:17:...|2014-04-02 13:30:...|             746.01|               3|
|      4|2014-04-07 12:09:...|2014-04-07 12:26:...|            1035.01|               2|
|      6|2014-04-06 16:58:...|2014-04-06 17:02:...|             246.01|               2|
+-------+--------------------+--------------------+-------------------+----------------+
only showing top 5 rows



In [104]:
df_sess.printSchema()

root
 |-- sess_id: integer (nullable = true)
 |-- start_ts: timestamp (nullable = true)
 |-- end_ts: timestamp (nullable = true)
 |-- total_duration_secs: double (nullable = true)
 |-- clicks_count_sum: integer (nullable = true)



### d. Find click  rate  (clicks_count_sum/total_duration_secs)

Skipped because I kept getting `SIGSEGV`.

In [105]:
df_sess = df_sess.withColumn('click_rate', F.col('clicks_count_sum') / (F.col('total_duration_secs')))

In [106]:
df_sess.show(5)

+-------+--------------------+--------------------+-------------------+----------------+--------------------+
|sess_id|            start_ts|              end_ts|total_duration_secs|clicks_count_sum|          click_rate|
+-------+--------------------+--------------------+-------------------+----------------+--------------------+
|      1|2014-04-07 10:51:...|2014-04-07 10:57:...|             351.01|               4|0.011395686732571722|
|      2|2014-04-07 13:56:...|2014-04-07 14:02:...|             359.01|               6|0.016712626389237067|
|      3|2014-04-02 13:17:...|2014-04-02 13:30:...|             746.01|               3|0.004021393815096312|
|      4|2014-04-07 12:09:...|2014-04-07 12:26:...|            1035.01|               2|0.001932348479724...|
|      6|2014-04-06 16:58:...|2014-04-06 17:02:...|             246.01|               2|0.008129750823137272|
+-------+--------------------+--------------------+-------------------+----------------+--------------------+
only showi

### e. Find distinct number of items viewed (total_items)

In [107]:
gdf_clicks = df_clicks_small.groupBy('sess_id')

In [108]:
counts = gdf_clicks.agg(F.countDistinct('item_id')).collect()

In [109]:
count_per_sess = {}
for c in counts:
    count_per_sess[c[0]] = c[1]

In [110]:
freq = F.udf(lambda sess_id: count_per_sess[sess_id], IntegerType())

In [111]:
df_sess = df_sess.withColumn('total_items', freq(df_sess.sess_id))
df_sess.show(5)

+-------+--------------------+--------------------+-------------------+----------------+--------------------+-----------+
|sess_id|            start_ts|              end_ts|total_duration_secs|clicks_count_sum|          click_rate|total_items|
+-------+--------------------+--------------------+-------------------+----------------+--------------------+-----------+
|      1|2014-04-07 10:51:...|2014-04-07 10:57:...|             351.01|               4|0.011395686732571722|          4|
|      2|2014-04-07 13:56:...|2014-04-07 14:02:...|             359.01|               6|0.016712626389237067|          5|
|      3|2014-04-02 13:17:...|2014-04-02 13:30:...|             746.01|               3|0.004021393815096312|          3|
|      4|2014-04-07 12:09:...|2014-04-07 12:26:...|            1035.01|               2|0.001932348479724...|          2|
|      6|2014-04-06 16:58:...|2014-04-06 17:02:...|             246.01|               2|0.008129750823137272|          2|
+-------+---------------

In [112]:
df_sess.printSchema()

root
 |-- sess_id: integer (nullable = true)
 |-- start_ts: timestamp (nullable = true)
 |-- end_ts: timestamp (nullable = true)
 |-- total_duration_secs: double (nullable = true)
 |-- clicks_count_sum: integer (nullable = true)
 |-- click_rate: double (nullable = true)
 |-- total_items: integer (nullable = true)



### f. Find distinct number of categories viewed (total_cats)

In [113]:
gdf_clicks = df_clicks_small.groupBy('sess_id')

In [114]:
counts = gdf_clicks.agg(F.countDistinct('category')).collect()

In [115]:
count_per_sess = {}
for c in counts:
    count_per_sess[c[0]] = c[1]

In [116]:
freq = F.udf(lambda sess_id: count_per_sess[sess_id], IntegerType())

In [117]:
df_sess = df_sess.withColumn('total_cats', freq(df_sess.sess_id))
df_sess.show(5)

+-------+--------------------+--------------------+-------------------+----------------+--------------------+-----------+----------+
|sess_id|            start_ts|              end_ts|total_duration_secs|clicks_count_sum|          click_rate|total_items|total_cats|
+-------+--------------------+--------------------+-------------------+----------------+--------------------+-----------+----------+
|      1|2014-04-07 10:51:...|2014-04-07 10:57:...|             351.01|               4|0.011395686732571722|          4|         1|
|      2|2014-04-07 13:56:...|2014-04-07 14:02:...|             359.01|               6|0.016712626389237067|          5|         1|
|      3|2014-04-02 13:17:...|2014-04-02 13:30:...|             746.01|               3|0.004021393815096312|          3|         1|
|      4|2014-04-07 12:09:...|2014-04-07 12:26:...|            1035.01|               2|0.001932348479724...|          2|         1|
|      6|2014-04-06 16:58:...|2014-04-06 17:02:...|             246.0

In [118]:
df_sess.printSchema()

root
 |-- sess_id: integer (nullable = true)
 |-- start_ts: timestamp (nullable = true)
 |-- end_ts: timestamp (nullable = true)
 |-- total_duration_secs: double (nullable = true)
 |-- clicks_count_sum: integer (nullable = true)
 |-- click_rate: double (nullable = true)
 |-- total_items: integer (nullable = true)
 |-- total_cats: integer (nullable = true)



### g. Find max dwell and min dwell  in a session

In [121]:
tdf = (df_clicks_small
.groupBy(F.col('sess_id'))
.agg(
    F.min('dwell_time').alias('min_dwell_time'),
    F.max('dwell_time').alias('max_dwell_time')
))

In [129]:
df_sess = df_sess.join(tdf, on=['sess_id'], how='left')

In [130]:
df_sess.limit(50).toPandas().head()

,sess_id,start_ts,end_ts,total_duration_secs,clicks_count_sum,click_rate,total_items,total_cats,min_dwell_time,max_dwell_time
0,1,2014-04-07 10:51:09.277,2014-04-07 10:57:00.306,351.01,4,0.011396,4,1,37,180
1,2,2014-04-07 13:56:37.614,2014-04-07 14:02:36.889,359.01,6,0.016713,5,1,42,118
2,3,2014-04-02 13:17:46.940,2014-04-02 13:30:12.318,746.01,3,0.004021,3,1,250,496
3,4,2014-04-07 12:09:10.948,2014-04-07 12:26:25.416,1035.01,2,0.001932,2,1,1035,1035
4,6,2014-04-06 16:58:20.848,2014-04-06 17:02:26.976,246.01,2,0.008130,2,1,246,246


### h. Replace all null/inf values with 0

In [131]:
df_sess = df_sess.fillna(0)

### i. Drop start_ts  and end_ts from the dataset

In [135]:
df_sess = df_sess.drop('start_ts').drop('end_ts')

In [136]:
df_sess.printSchema()

root
 |-- sess_id: integer (nullable = true)
 |-- total_duration_secs: double (nullable = false)
 |-- clicks_count_sum: integer (nullable = true)
 |-- click_rate: double (nullable = false)
 |-- total_items: integer (nullable = true)
 |-- total_cats: integer (nullable = true)
 |-- min_dwell_time: long (nullable = true)
 |-- max_dwell_time: long (nullable = true)



### j. Using buys datasets, find where or not the user buys in a given session. [If a session id is present in the buys dataset that indicates the user has made a purchase]. Create a new column (buy = 0: not buy, 1: buy)

Needed to add a constant column of '1' to the buys dataset and perform a left inner join. Could not finish because the kernel kept dying intermittently.

## Unattemped Questions

Using buys datasets, find how many items are purchased [items_bought]
Create one-hot-encoding the category column on clicks dataset (add one column for each category -  value is 1/0 based on where or not the record belongs the category)
Using the start time of the session, find the hour of the day for each session start.  
Using session dataset created above, create a pivot table for conversion ratio by hour and category
rows => item categories
columns => hour of the day
values => conversion ratio = number sessions with buys / total number of sessions
Using session dataset created above, create a pivot table for conversion ratio by hour and category
rows => item categories
columns => hour of the day
values => average price of buys
